# Actividad 4.3. Similitud en textos mediante TF-IDF y Cadenas de Markov
---
**Por: Ian Joab Padron Corona - A01708940**

In [25]:
import pandas as pd
import math
import re

In [26]:
def df_file(archivo):
    data = pd.read_csv(archivo, header=0, encoding='utf-8')
    return data

In [27]:
def clean_text(text):
    if not isinstance(text, str):
        text = str(text)
    text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)
    return text.lower()

In [28]:
def array_words(text):
    return text.split()

In [29]:
def uniqueWords(q1, q2):
    words_q1 = array_words(q1)
    words_q2 = array_words(q2)
    unique_words = set(words_q1 + words_q2)
    return list(unique_words)

In [30]:
def countWords(q, wordsq1q2):
    q_words = array_words(q)
    wordsCount = []
    for word in wordsq1q2:
        wordsCount.append(q_words.count(word)/len(q_words))
    return wordsCount

In [31]:
def idf(words, q1, q2):
    idfs = []
    for word in words:
        docs_count = 0
        if word in array_words(q1):
            docs_count += 1
        if word in array_words(q2):
            docs_count += 1
        idf = ((math.log(2/(docs_count + 1))) + 1)
        idfs.append(idf)
    return idfs

In [32]:
def tfByIDF(tfQ, idf):
    return [tf * idf_val for tf, idf_val in zip(tfQ, idf)]

In [ ]:
def main():
    # Definir el nombre del archivo
    archivo = 'questions.csv'
    
    # Llamar la funcion df_file para obtener el DataFrame
    data = df_file(archivo).head(3)
    
    # Aplicar la funcion regex a la columna 'question1' y 'question2 del DataFrame
    data['question1'] = data['question1'].apply(clean_text)
    data['question2'] = data['question2'].apply(clean_text)

    # Crear una columna que contenga un array con todas las palabras de question1 y question2, sin duplicados
    data['words'] = data.apply(lambda x: uniqueWords(x['question1'], x['question2']), axis=1)

    # Crear una columna que contenga un array con la cantidad de veces que aparece cada palabra de words en q1,q2
    data['TF_q1'] = data.apply(lambda x: countWords(x['question1'], x['words']), axis=1)
    data['TF_q2'] = data.apply(lambda x: countWords(x['question2'], x['words']), axis=1)

    # Crear una columna que contenga el array de IDF's de cada palabra en words
    data['IDF'] = data.apply(lambda x: idf(x['words'], x['question1'], x['question2']), axis=1)

    # Crear una columna que contenga el array de TF-IDF's de cada palabra en words
    data['TF-IDFq1'] = data.apply(lambda x: tfByIDF(x['TF_q1'], x['IDF']), axis=1)
    data['TF-IDFq2'] = data.apply(lambda x: tfByIDF(x['TF_q2'], x['IDF']), axis=1)
    
    # Guardar el DataFrame en un archivo CSV
    data.to_csv('tf_IDF.csv', index=False)


main()